# A Blitz Introduction to DGL - Node Classification

GNNs are powerful tools for many machine learning tasks on graphs.  In this introductory tutorial, you will learn the basic workflow of using GNNs for node classification, i.e. predicting the category of a node in a graph.

Goal of this tutorial:

* Train a node classification neural network on a single small graph.

This tutorial assumes that you have experience in building neural networks with PyTorch.

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


## Overview of Node Classification with GNN

Many proposed methods are *unsupervised* (or *self-supervised* by recent definition), where the model predicts the community labels only by connectivity. Recently, [Kipf et al.,](https://arxiv.org/abs/1609.02907) proposed to formulate the community detection problem as a node classification task. With the help of only a small portion of labeled nodes, a graph neural network (GNN) can accurately predict the community labels of the others.

This tutorial will show how to build such a GNN for semi-supervised node classification with only a small number of labels on [Cora dataset](https://docs.dgl.ai/api/python/dgl.data.html#dgl.data.CoraGraphDataset), a citation network with papers as nodes and citations as edges.  The task is to predict the category of a given paper.  The papers contain word count vectorization as features, normalized so that they sum up to 1, as in Section 5.2 in [the paper](https://arxiv.org/abs/1609.02907).

## Loading Cora Dataset

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


A DGL Dataset object may contain one or multiple graphs.  The Cora dataset used in this tutorial only consists of one single graph.

In [3]:
g = dataset[0]

DGL graphs can store node-wise and edge-wise information in `ndata` and `edata` attribute as dictionaries.  In the DGL Cora dataset, the graph contains:

* `train_mask`: Whether the node is in training set.
* `val_mask`: Whether the node is in validation set.
* `test_mask`: Whether the node is in test set.
* `label`: The ground truth node category.
* `feat`: The node features.

In [4]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


## Define a Graph Convolutional Network (GCN)

This tutorial will build a two-layer [Graph Convolutional Network (GCN)](http://tkipf.github.io/graph-convolutional-networks/).  Each of its layer computes new node representations by aggregating neighbor information.

To build a multi-layer GCN you can simply stack `dgl.nn.GraphConv` modules, which inherit `torch.nn.Module`.

In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

DGL provides implementation of many popular neighbor aggregation modules.  They all can be invoked easily with one line of code.

## Training the GCN

Training this GCN is similar to training other PyTorch neural networks.

In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.132 (best 0.132), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.892, val acc: 0.516 (best 0.540), test acc: 0.546 (best 0.556)
In epoch 10, loss: 1.811, val acc: 0.684 (best 0.684), test acc: 0.705 (best 0.702)
In epoch 15, loss: 1.705, val acc: 0.724 (best 0.724), test acc: 0.723 (best 0.723)
In epoch 20, loss: 1.574, val acc: 0.732 (best 0.740), test acc: 0.741 (best 0.734)
In epoch 25, loss: 1.421, val acc: 0.738 (best 0.740), test acc: 0.739 (best 0.734)
In epoch 30, loss: 1.251, val acc: 0.744 (best 0.746), test acc: 0.743 (best 0.735)
In epoch 35, loss: 1.074, val acc: 0.756 (best 0.756), test acc: 0.745 (best 0.745)
In epoch 40, loss: 0.900, val acc: 0.752 (best 0.756), test acc: 0.746 (best 0.745)
In epoch 45, loss: 0.739, val acc: 0.754 (best 0.756), test acc: 0.746 (best 0.745)
In epoch 50, loss: 0.599, val acc: 0.760 (best 0.760), test acc: 0.748 (best 0.748)
In epoch 55, loss: 0.481, val acc: 0.764 (best 0.764), test acc: 0.751 (best 0

## Training on GPU

Training on GPU requires to put both the model and the graph onto GPU with the `to` method, similar to what you will do in PyTorch.

In [7]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.252 (best 0.252), test acc: 0.224 (best 0.224)
In epoch 5, loss: 1.896, val acc: 0.432 (best 0.452), test acc: 0.476 (best 0.472)
In epoch 10, loss: 1.821, val acc: 0.484 (best 0.484), test acc: 0.503 (best 0.503)
In epoch 15, loss: 1.723, val acc: 0.558 (best 0.558), test acc: 0.572 (best 0.572)
In epoch 20, loss: 1.605, val acc: 0.622 (best 0.622), test acc: 0.635 (best 0.635)
In epoch 25, loss: 1.468, val acc: 0.656 (best 0.656), test acc: 0.658 (best 0.658)
In epoch 30, loss: 1.318, val acc: 0.676 (best 0.676), test acc: 0.683 (best 0.681)
In epoch 35, loss: 1.161, val acc: 0.696 (best 0.696), test acc: 0.703 (best 0.703)
In epoch 40, loss: 1.004, val acc: 0.712 (best 0.712), test acc: 0.711 (best 0.711)
In epoch 45, loss: 0.853, val acc: 0.722 (best 0.722), test acc: 0.727 (best 0.727)
In epoch 50, loss: 0.715, val acc: 0.736 (best 0.736), test acc: 0.739 (best 0.739)
In epoch 55, loss: 0.594, val acc: 0.752 (best 0.752), test acc: 0.744 (best 0

## What's next?

* [How does DGL represent a graph](2_dglgraph.ipynb)?
* [Write your own GNN module](3_message_passing.ipynb).
* [Link prediction (predicting existence of edges) on full graph](4_link_predict.ipynb).
* [Graph classification](5_graph_classification.ipynb).
* [Make your own dataset](6_load_data.ipynb).
* [The list of supported graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv).

### Scale to large graphs
* If you wish to scale your model to a large graph, please begin with the tutorial [Stochastic Training of GNN for Node Classification on Large Graphs](L1_large_node_classification.ipynb).

### Advanced topics

* [Running with multiple GPUs (TODO)]()
  * [Large graph]()
  * [Graph classification]()
* [Running with distributed training (TODO)]()

<!--
### Other materials
* [DGL examples](I1_examples.ipynb): this is a catalog of all end-to-end examples provided by DGL.
* [HOWTOs](I2_tutorials.ipynb): this is a catalog of individual articles describing how to build models with DGL for various data and scenarios.
-->